# Simple Index Demo

#### Load documents, build the GPTSimpleVectorIndex

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [2]:
from gpt_index.readers import JSONReader

In [20]:
docs = JSONReader('test_json/raw.json', levels_back=0).load_data()

In [21]:
doc = docs[0]
print(doc.get_text())

buildId 2023-02-22 fc5f219ad
content additionalAttributes attributes condition categories smartphones
content additionalAttributes attributes condition categories smartwatches
content additionalAttributes attributes condition categories tablets
content additionalAttributes attributes condition categories computing
content additionalAttributes attributes condition categories carrierstores
content additionalAttributes attributes condition categories dynamic
content additionalAttributes attributes condition categories hubs
content additionalAttributes attributes condition country us
content additionalAttributes attributes values selectAllRelation_bk True
content additionalAttributes attributes values addPCToPricing_delete True
content additionalAttributes attributes values summaryView_bk True
content additionalAttributes attributes values headerCTA_bk True
content additionalAttributes attributes values showWhereToBuy_bk False
content additionalAttributes attributes values excludeNonCarrie

In [ ]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [ ]:
index = GPTSimpleVectorIndex(documents)

In [ ]:
# save index to disk
index.save_to_disk('index_simple.json')

In [ ]:
# load index from disk
index = GPTSimpleVectorIndex.load_from_disk('index_simple.json')

#### Query Index

In [ ]:
# set Logging to DEBUG for more detailed outputs
response = index.query("What did the author do growing up?")

In [ ]:
display(Markdown(f"<b>{response}</b>"))

**Query Index with custom embedding string**

In [ ]:
from gpt_index.indices.query.schema import QueryBundle

In [ ]:
query_bundle = QueryBundle(
    query_str="What did the author do growing up?", 
    custom_embedding_strs=['The author grew up painting.']
)
response = index.query(query_bundle)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

#### Get Sources

In [ ]:
print(response.get_formatted_sources())